In [3]:
import requests
import re
from bs4 import BeautifulSoup 
import time
from datetime import datetime
from selenium import webdriver
import pandas as pd
import numpy as np
import csv

In [14]:
# 본인 로컬 환경에 맞게 변경
browser = webdriver.Chrome('C:/Users/qudwn/OneDrive/바탕 화면/PythonWorkspace/chromedriver_win32/chromedriver.exe')
browser.get("https://tickets.interpark.com/goods/23001653")
time.sleep(3)

In [15]:
browser.find_element_by_xpath('//*[@id="popup-prdGuide"]/div/div[3]/button').click()
time.sleep(1)

In [16]:
browser.find_element_by_xpath('//*[@id="productMainBody"]/nav/div/div/ul/li[4]/a').click()
time.sleep(1)

In [23]:
# 본인 뮤지컬 페이지에 맞게 변경, 페이지 넘어가는 것은 for 문으로 수정해서 진행하면 될 것 같고 한가지 문제가 review_text 
# 내용을 전체를 못가져오고 첫줄만 가져오는데 혹시 방법 찾으면 공유 부탁해 ㅠㅠ
columns = ['rate', 'review_title', 'review_text', 'review_date']
values = []
soup = BeautifulSoup(browser.page_source, 'lxml')

data_rows = soup.find_all('li', attrs={'class':'bbsItem'})
for row in data_rows:
    blank = []
    rate = row.find('div', attrs={'class':'prdStarIcon'})
    if rate:
        rate = rate['data-star'].strip()
        blank.append(rate)
    else:
        #print("<평점 없는 리뷰 제외합니다>")
        continue
    
    review_title = row.find('strong', attrs={'class':'bbsTitleText'})
    if review_title:
        review_title = review_title.get_text()
        blank.append(review_title)
    else:
        #print("<리뷰제목 없는 리뷰 제외합니다>")
        continue

    review_text = row.find('p', attrs={'class':'bbsText'})
    if review_text:
        review_text = review_text.get_text().strip()
        blank.append(review_text)
    else:
        #print("<리뷰내용 없는 리뷰 제외합니다>")
        continue
    
    date = row.find_all('li', attrs={'class':'bbsItemInfoList'})[1]
    if date:
        date = date.get_text()
        blank.append(date)
    else:
        #print("<작성날짜 없는 리뷰 제외합니다>")
        continue
    
    values.append(blank)


df = pd.DataFrame(values, columns = columns)
df['musical_title'] = '영웅'
df = df[['musical_title', 'rate', 'review_title', 'review_text', 'review_date']]
df.head()

,musical_title,rate,review_title,review_text,review_date
0,영웅,5,뮤지컬영웅은 사랑입니다♡,아이가 영웅에 빠져서 또 보고싶대요,2023.05.05
1,영웅,5,위국헌신 군인본분,"작년 영화로 처음 접했고, 이번에 뮤지컬을 통하여 전혀 새로운 경험을 했습니다.역시...",2023.05.05
2,영웅,5,영웅,친구가 연주하는오케스트라에 제일좋아하는 뮤지컬배우정성화님... :) 눈호강 귀호강 ...,2023.05.05
3,영웅,5,앙상블들이 아주 멋있고 뛰어났습니다,정성화 배우야 말해 뭐하나 싶게 뛰어난 연기 노래로 감동을 줬고 앙상블 배우들의 춤...,2023.05.05
4,영웅,5,네 번째 관람 <영웅>,"1층 2열 중앙에서 본이번 공연 또한 전율, 오열, 감동이었습니다.정성화 배우님, ...",2023.05.05


In [24]:
browser.close()